# Team Assignment 4

Team Sakia: Iara Ravagni and Sakshee Patil

Date: 09-24-2024

## Import libraries

In [7]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

## Download CSV Files

This step downloads the dataset from the original [GitHub repository](https://github.com/Welltory/hrv-covid19/tree/master). The dataset includes the following CSV files:

- participants.csv
- sleep.csv
- scales_description.csv
- weather.csv
- blood_pressure.csv
- hrv_measurements.csv
- surveys.csv
- wearables.csv
- heart_rate.csv

In [15]:
# Define the URL of the GitHub data folder
url = 'https://github.com/Welltory/hrv-covid19/tree/master/data'

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links to CSV files
csv_links = set()  # Use a set to avoid duplicates
for link in soup.find_all('a'):
    href = link.get('href')
    if href and href.endswith('.csv'):
        # Construct the raw URL and add to the set
        raw_url = f'https://raw.githubusercontent.com{href.replace("tree/master", "master")}'
        csv_links.add(raw_url)

# Create a directory to save the CSV files
os.makedirs('data', exist_ok=True)

# Download each CSV file
for csv_link in csv_links:
    filename = os.path.join('data', csv_link.split('/')[-1])
    with requests.get(csv_link) as r:
        with open(filename, 'wb') as f:
            f.write(r.content)
    print(f'Downloaded: {filename}')

Downloaded: data\participants.csv
Downloaded: data\sleep.csv
Downloaded: data\scales_description.csv
Downloaded: data\weather.csv
Downloaded: data\blood_pressure.csv
Downloaded: data\hrv_measurements.csv
Downloaded: data\surveys.csv
Downloaded: data\wearables.csv
Downloaded: data\heart_rate.csv
